<a href="https://colab.research.google.com/github/epodkwan/growthfunction/blob/main/gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 202 kB 3.2 MB/s 
     |████████████████████████████████| 9.1 MB 8.5 MB/s 
     |████████████████████████████████| 145 kB 43.3 MB/s 
     |████████████████████████████████| 217 kB 43.5 MB/s 
     |████████████████████████████████| 596 kB 11.5 MB/s 
     |████████████████████████████████| 51 kB 8.3 MB/s 
     |████████████████████████████████| 72 kB 711 kB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from typing import Sequence
import jax
import optax
import numpy as np
import jax.numpy as jnp
from jax import jit
from flax import linen as nn
from flax.training import train_state,checkpoints

In [ ]:
def npy_loader(path):
    return jnp.load(path)

In [ ]:
class SimpleMLP(nn.Module):
    features:Sequence[int]

    @nn.compact
    def __call__(self,inputs):
        x=inputs
        for i,feat in enumerate(self.features):
            x=nn.Dense(feat)(x)
            if i != len(self.features)-1:
                x=nn.relu(x)
        return x

In [ ]:
layer_sizes=[64,256,256,256]
learning_rate=1e-6
model=SimpleMLP(features=layer_sizes)
temp=jnp.ones(2)
params=model.init(jax.random.PRNGKey(0),temp)
tx=optax.adam(learning_rate=learning_rate)
opt_state=tx.init(params)
state=train_state.TrainState.create(apply_fn=model.apply,params=params,tx=tx)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
restored_state=checkpoints.restore_checkpoint(ckpt_dir="/content/drive/My Drive/Colab Notebooks/checkpoint_0",target=state)
cosmo=npy_loader("/content/drive/My Drive/Colab Notebooks/cosmo.npy")
z=npy_loader("/content/drive/My Drive/Colab Notebooks/999.npy")[0,:]
a=1/(z+1)

In [ ]:
@jit
def predict(params,x,j):
    pred=jnp.exp(restored_state.apply_fn(params,x))
    return pred[j]

In [ ]:
@jit
def gradient_at(i,j):
    value,gradient=jax.value_and_grad(predict,1)(restored_state.params,jnp.array([cosmo[i,0],cosmo[i,2]]),j)
    test1=background.MatterDominated(cosmo[i,0],1-cosmo[i,0],0,a=None,a_normalize=1.0)
    test2=background.MatterDominated(cosmo[i,0]+0.001,1-cosmo[i,0]-0.001,0,a=None,a_normalize=1.0)
    arr=[(test2.D1(a[j])-test1.D1(a[j]))/0.001]
    return gradient

In [ ]:
for j in range(256):
    gradient=gradient_at(0,j)
    print(gradient)
drive.flush_and_unmount()

[-0.00834076 -0.00126369]
[-0.00536724 -0.00066295]
[-0.02702514 -0.00162067]
[-0.00525265 -0.00037597]
[-0.03170329 -0.00381757]
[-0.01027681 -0.001058  ]
[-0.01210345 -0.00150825]
[-0.0405968 -0.0039377]
[-0.02044143 -0.00070631]
[-0.00962581 -0.00055232]
[-0.01862828 -0.00079207]
[-0.01708093 -0.00121806]
[-0.01875655 -0.00106637]
[-0.01938896 -0.00201891]
[-0.05921948 -0.00780117]
[-0.01039953 -0.00107063]
[-0.04298759 -0.00554679]
[-0.04420489 -0.00229748]
[-0.01925214 -0.00471864]
[-0.01595523 -0.00292109]
[-0.04526237 -0.00277239]
[-0.01764586 -0.00084071]
[-0.01046069 -0.00181805]
[-0.00778625 -0.00073618]
[-0.00745014 -0.00119076]
[-0.00572165 -0.00122056]
[-0.007509   -0.00120016]
[-0.00963371 -0.00053578]
[-0.0188635  -0.00107245]
[-0.00528085 -0.00037799]
[-0.01098159 -0.0006581 ]
[-0.00831081 -0.00125916]
[-0.00539123 -0.00066592]
[-0.01101818 -0.00066029]
[-0.01115167 -0.00066829]
[-0.00648749 -0.00096421]
[-0.03508119 -0.00176412]
[-0.0093242  -0.00227793]
[-0.01781016 -